+ Pixelized shear measurements (smoothing -> pixelize/ pixelize -> smoothing)
+ Get the variance map (mock/analytical)

In [1]:
%pylab inline
import os
import gc
import numpy as np
import astropy.io.fits as pyfits
import maputil as mutilities
from configparser import ConfigParser
from pixel3D import cartesianGrid3D

Populating the interactive namespace from numpy and matplotlib


In [2]:
configName  =   'wrkDir/config-nl15.ini'
parser      =   ConfigParser()
parser.read(configName)
outDir      =   'wrkDir/galmap3D/'

In [8]:
for fieldname in mutilities.field_names:
    infname='wrkDir/reGaus_catalog/%s.fits' %(fieldname)
    data=pyfits.getdata(infname)
    if fieldname=='VVDS':
        addR=(data['ra']<300.).astype(int)*360
        data['ra']+=addR
    raMin=(np.min(data['ra'])-1.)*60.
    raMax=(np.max(data['ra'])+1.)*60.
    decMin=(np.min(data['dec'])-1.)*60.
    decMax=(np.max(data['dec'])+1.)*60.
    nx=int((raMax-raMin+1)//2*2)
    ny=int((decMax-decMin+1)//2*2)
    print(raMin,decMin)
    parser.set('transPlane','xmin','%s'%raMin)
    parser.set('transPlane','ymin','%s'%decMin)
    parser.set('transPlane','nx','%d'%nx)
    parser.set('transPlane','ny','%d'%ny)
    gridInfo    =   cartesianGrid3D(parser)
    x,y,z=data['ra'],data['dec'],data['dnn_zbest']
    ws=data['weight']
    # w*g1,w*g2,w for each pixel
    weightOutTmp=numpy.histogramdd((z,y,x),bins=(gridInfo.zlbound,gridInfo.ybound,gridInfo.xbound))[0]
    mask0=weightOutTmp>0. # the mask before smoothing
    pyfits.writeto(os.path.join(outDir,'%s_dnnz_10_bins_numMap.fits' %fieldname),weightOut,overwrite=True)
    del weightOut
    gc.collect()

1685.6270882643867 -443.45298558012774
7631.884040200001 -173.94776607479398
9176.585232079698 -160.59038293279627
12306.362086594861 -162.72370495177304
19768.172504524035 -120.3134553150722
12694.431102030565 2472.7539675342996
